# Códigos Examen de Algoritmos Bioinspirados 
# BLOQUE 4
Chiara Bombardieri Balanzá - A01659462

In [1]:
import numpy as np

In [6]:
# Bloques de bits con su contribución 
_CONTRIB = {
    '000': 2.8,
    '001': 2.6,
    '010': 2.2,
    '011': 0.0,
    '100': 1.4,
    '101': 0.0,
    '110': 0.0,
    '111': 3.0,
}

def fitness_engañoso(bits):
    n = bits.size
    total = 0.0
    for i in range(0, n, 3):
        bloque = ''.join('1' if b else '0' for b in bits[i:i+3])
        total += _CONTRIB[bloque]
    return total

# Algoritmo genético simple 
def ga_engañoso(n,
                pop_size,
                tasa_cruce,
                tasa_mut,
                generaciones,
                seed=None,
                elite=1,
                torneo_k=3,
                verbose=False):
    assert n % 3 == 0, "n debe ser múltiplo de 3 (bloques de 3 bits)."
    rng = np.random.default_rng(seed)

    # Población inicial 0/1
    pop = rng.integers(0, 2, size=(pop_size, n), dtype=np.int8)

    def eval_pop(P):
        return np.array([fitness_engañoso(ind) for ind in P], dtype=float)

    fit = eval_pop(pop)
    max_fit = float(n)  # óptimo global
    best_idx = int(np.argmax(fit))
    best = pop[best_idx].copy()
    best_fit = float(fit[best_idx])

    for g in range(1, generaciones + 1):
        # Elitismo
        elite_idx = np.argsort(-fit)[:elite]
        elites = pop[elite_idx].copy()

        # Selección por torneo
        def sel():
            idxs = rng.integers(0, pop_size, size=torneo_k)
            return pop[idxs[np.argmax(fit[idxs])]]

        nueva = np.empty_like(pop)
        i = 0
        if elite > 0:
            nueva[:elite] = elites
            i = elite

        # Reproducción
        while i < pop_size:
            p1 = sel()
            p2 = sel()

            # Cruce de un punto
            if rng.random() < tasa_cruce:
                punto = rng.integers(1, n)
                h1 = np.concatenate([p1[:punto], p2[punto:]])
                h2 = np.concatenate([p2[:punto], p1[punto:]])
            else:
                h1, h2 = p1.copy(), p2.copy()

            # Mutación bit-flip independiente por bit
            if tasa_mut > 0.0:
                m1 = rng.random(n) < tasa_mut
                m2 = rng.random(n) < tasa_mut
                h1[m1] ^= 1
                h2[m2] ^= 1

            nueva[i] = h1
            if i + 1 < pop_size:
                nueva[i + 1] = h2
            i += 2

        pop = nueva
        fit = eval_pop(pop)

        # Actualiza mejor
        idx = int(np.argmax(fit))
        if fit[idx] > best_fit:
            best_fit = float(fit[idx])
            best = pop[idx].copy()

        if verbose and (g % max(1, generaciones // 10) == 0 or best_fit >= max_fit):
            print(f"Gen {g:4d} | Mejor aptitud = {best_fit:.4f}")

        if best_fit >= max_fit:
            break

    mejor_cadena = ''.join('1' if b else '0' for b in best)
    return mejor_cadena, best_fit

def correr(n, pop_size, tasa_cruce, tasa_mut, generaciones,
           seed=42, verbose=False):
    mejor, apt = ga_engañoso(n, pop_size, tasa_cruce, tasa_mut, generaciones, seed=seed, verbose=verbose)
    print("Mejor solución:", mejor)
    print("Aptitud:", round(apt, 4))


Inputs para que se use el código 

In [9]:
def _leer_int(mensaje, cond=None, msg_error="Valor inválido."):
    while True:
        s = input(mensaje).strip()
        try:
            v = int(s)
            if cond is None or cond(v):
                return v
        except:
            pass
        print(msg_error)

def _leer_float(mensaje, cond=None, msg_error="Valor inválido."):
    while True:
        s = input(mensaje).strip()
        try:
            v = float(s)
            if cond is None or cond(v):
                return v
        except:
            pass
        print(msg_error)

def correr_desde_input():
    print("Problema engañoso por bloques de 3 bits ===")
    n = _leer_int("Número de bits (múltiplo de 3): ",
                  cond=lambda x: x > 0 and x % 3 == 0,
                  msg_error="Debe ser entero > 0 y múltiplo de 3.")
    pop_size = _leer_int("Tamaño de la población: ",
                         cond=lambda x: x >= 2,
                         msg_error="Debe ser entero >= 2.")
    tasa_cruce = _leer_float("Tasa de cruce (0 a 1): ",
                             cond=lambda x: 0.0 <= x <= 1.0,
                             msg_error="Debe estar entre 0 y 1.")
    tasa_mut = _leer_float("Tasa de mutación (0 a 1): ",
                           cond=lambda x: 0.0 <= x <= 1.0,
                           msg_error="Debe estar entre 0 y 1.")
    generaciones = _leer_int("Número de generaciones: ",
                             cond=lambda x: x > 0,
                             msg_error="Debe ser entero > 0.")
    seed_txt = input("Semilla aleatoria (opcional, Enter para aleatoria): ").strip()
    seed = int(seed_txt) if seed_txt else None

    print("\nEjecutando...\n")
    mejor, apt = ga_engañoso(n, pop_size, tasa_cruce, tasa_mut, generaciones, seed=seed, verbose=True)
    print("\n=== Resultado ===")
    print("Mejor solución:", mejor)
    print("Aptitud:", round(apt, 4))

# Ejecuta modo interactivo 
correr_desde_input()


Problema engañoso por bloques de 3 bits ===

Ejecutando...

Gen   50 | Mejor aptitud = 14.8000
Gen  100 | Mejor aptitud = 14.8000
Gen  150 | Mejor aptitud = 14.8000
Gen  198 | Mejor aptitud = 15.0000

=== Resultado ===
Mejor solución: 111111111111111
Aptitud: 15.0
